In [1]:
from confluent_kafka import Consumer, KafkaError, Producer
import json
from time import sleep
import numpy as np
from collections import defaultdict
from CountMin import *
from FlajoletMartin import *
from Kmeans import *

# Global variables
consumer = None
counter = 0
num_data = 0
bootstrap_servers = 'kafka:9092'
input_topic = 'Epsymolo'
output_topic = 'Epsymolo1'


# Kafka Consumer and Producer functions
def create_consumer(bootstrap_servers, topic, timeout=10.0):
    conf = {
        'bootstrap.servers': bootstrap_servers,
        'group.id': 'kafka_python_consumer',
        'auto.offset.reset': 'earliest'
    }
    consumer = Consumer(conf)
    consumer.subscribe([topic])
    return consumer

def create_producer(bootstrap_servers):
    producer = Producer({'bootstrap.servers': bootstrap_servers})
    return producer

# Message processing functions
def consume_write(message_value):
    global num_data
    num_data += 1
    print(num_data, message_value)

def produce_to_kafka(message_value, results):
      # Add the results to the message
    for key, value in results.items():
        message_value[key] = value

    # Send the modified message to the Kafka producer
    producer.produce(topic=output_topic, value=json.dumps(message_value).encode('utf-8'))


# Main logic
while counter < 10 and consumer is None:
    try:
        print('Trying to connect to Kafka...')
        consumer = create_consumer(bootstrap_servers, input_topic, timeout=10.0)
    except:
        print('Kafka is not reachable yet.')
        counter += 1
        sleep(10)

if consumer is not None:
    print('Kafka is up.')
    producer = create_producer(bootstrap_servers)
    print('Printing messages on topic Epsymolo1...')

    try:
        while True:
            timeout = 1.0
            msg = consumer.poll(timeout)  # Poll for messages
            if msg is None:
                timeout -= 1.0
                if timeout <= 0:
                    print("No more messages. Exiting...")
                    break
                else:
                    print(f"Waiting for messages. Timeout in {int(timeout)} seconds.")
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    print('End of partition reached {0}/{1}'.format(msg.topic(), msg.partition()))
                else:
                    print(f"Error: {msg.error()}")
            else:
                timeout = 10.0
                # Proper message
                message_value = json.loads(msg.value().decode('utf-8'))
            
                consume_write(message_value)
                
                # Aggregate results from different functions
                results = {
                'flajolet_martin': flajolet_martin(message_value),
                'Count_Min':count_min(message_value),
                'Cluster': clustering(message_value)
                
            }

                # Send the aggregated results to the output topic
                produce_to_kafka(message_value, results)
                # print(f"Received message: {msg.value().decode('utf-8')}")

    finally:
        consumer.close()
        producer.flush()
        # producer.close()
        print("Consumer and producer closed.")
        print('-----')

    print('Consumer process successfully ended.')

else:
    print('Consumer cannot be initialized. Exiting.')


Trying to connect to Kafka...
Kafka is up.
Printing messages on topic Epsymolo1...
1 {'Timestamp': 1702996703906, 'DayOfWeek': 'Friday', 'PowerLineID': 949.0, 'PowerFlowValue': -133.93}
2 {'Timestamp': 1702996708912, 'DayOfWeek': 'Friday', 'PowerLineID': 1687.0, 'PowerFlowValue': 297.0}
3 {'Timestamp': 1702996713913, 'DayOfWeek': 'Friday', 'PowerLineID': 1375.0, 'PowerFlowValue': 0.0}
4 {'Timestamp': 1702996718914, 'DayOfWeek': 'Friday', 'PowerLineID': 299.0, 'PowerFlowValue': -171.42}
5 {'Timestamp': 1702996723916, 'DayOfWeek': 'Friday', 'PowerLineID': 492.0, 'PowerFlowValue': -71.33}
6 {'Timestamp': 1702996728917, 'DayOfWeek': 'Friday', 'PowerLineID': 515.0, 'PowerFlowValue': 57.96}
7 {'Timestamp': 1702996733922, 'DayOfWeek': 'Friday', 'PowerLineID': 1206.0, 'PowerFlowValue': -51.55}
8 {'Timestamp': 1702996738922, 'DayOfWeek': 'Friday', 'PowerLineID': 1342.0, 'PowerFlowValue': 0.0}
9 {'Timestamp': 1702996743923, 'DayOfWeek': 'Friday', 'PowerLineID': 1503.0, 'PowerFlowValue': -59.24}


KeyboardInterrupt: 